In [1]:
import os, json, re
import random, time
import pandas as pd, numpy as np
from tqdm import tqdm
from datetime import datetime, timedelta
from dotenv import load_dotenv

from itertools import islice

# Util built for sharktank project
import trial_sharktank_utils_mod as su
from trial_sharktank_utils_mod import load_facts, PitchOrchestrator, metrics_calculation, PitchEditor, OrchestratorResponse, PitchEquipped

# Agno for llm agents
from agno.agent import Agent
from agno.models.groq import Groq
from agno.storage.agent.sqlite import SqliteAgentStorage
from agno.tools.duckduckgo import DuckDuckGoTools
# from agno.tools.wikipedia import WikipediaTools

# Other LLM utils
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List, Dict, Optional

c:\Users\gabjj\anaconda3\envs\sharktank\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



[Knowledge Loader] ================================
>> Chunking Strategy: FIXED
>> Agent Storage File: tmp/pdf_rag.db
>> Agent Table Name: HBS_fixed_agent
>> ChromaDB Collection Name: HBS_fixed
>> ChromaDB Storage Path: ./chromadb_data
>> PDFs Being Processed:
   - https://raw.githubusercontent.com/sciencenerd880/LLM-DBT/main/data/pdfs/hbs_opportunities.pdf
   - https://raw.githubusercontent.com/sciencenerd880/LLM-DBT/main/data/pdfs/hbs_pitchdeck_sample.pdf



INFO     Embedder not provided, using OpenAIEmbedder as default.

In [ ]:
load_dotenv()
os.environ["OEPNAI_API_KEY"] = ""
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [3]:
reference_models = su.REFERENCE_MODELS
agent_storage = su.agent_storage
edit_reference_models = su.EDIT_REFERENCE_MODELS

In [4]:
facts_store = load_facts()
pitches = {k:"" for k,_ in facts_store.items()}
metrics = {k:"" for k,_ in facts_store.items()}
time_stamps = {k:"" for k,_ in facts_store.items()}

In [5]:
# facts_store = load_facts()
# remaining_pitches = [
#     'facts_shark_tank_transcript_17_Kettle Gryp.txt',
#     'facts_shark_tank_transcript_25_Plufl.txt',
#     'facts_shark_tank_transcript_28_HummViewer.txt',
#     'facts_shark_tank_transcript_37_Krapp Strapp.txt'
# ]

# facts_store = {k:v for k,v in facts_store.items() if k in remaining_pitches}

In [6]:
print(facts_store.keys())

dict_keys(['facts_shark_tank_transcript_0_GarmaGuard.txt', 'facts_shark_tank_transcript_0_Roadie.txt', 'facts_shark_tank_transcript_0_TouchUp Cup.txt', 'facts_shark_tank_transcript_10_Deux.txt', 'facts_shark_tank_transcript_10_Fish Fixe.txt', 'facts_shark_tank_transcript_10_Hidrent.txt', 'facts_shark_tank_transcript_11_Dude Wipes.txt', 'facts_shark_tank_transcript_11_Love & Pebble.txt', 'facts_shark_tank_transcript_11_Pink Picasso.txt', 'facts_shark_tank_transcript_11_Sheets Laundry Club.txt', 'facts_shark_tank_transcript_11_Zach & Zoe Sweet Bee Farm.txt', 'facts_shark_tank_transcript_12_Elf Grams.txt', 'facts_shark_tank_transcript_12_Ornament Anchor.txt', "facts_shark_tank_transcript_12_Santa's Enchanted Mailbox.txt", "facts_shark_tank_transcript_12_Wendy's Gnome Shop.txt", 'facts_shark_tank_transcript_13_Banana Loca.txt', 'facts_shark_tank_transcript_13_Liberate.txt', 'facts_shark_tank_transcript_13_MAXPRO SmartConnect.txt', 'facts_shark_tank_transcript_13_Tenikle.txt', 'facts_shark_

In [7]:
# Main loop
orcheditor = PitchEquipped(
    reference=edit_reference_models, 
    iterations=2,
    orchestrator="deepseek-r1-distill-llama-70b"
)
goal = 'create a pitch with the given facts'

RUN_FIRST_N = False      # Change to False to run all
N = 1

cases_to_run = islice(facts_store.items(), N) if RUN_FIRST_N else facts_store.items()
# cases_to_run = facts_store
for case, facts in tqdm(cases_to_run, desc="Generating pitches"):
# case = 'facts_shark_tank_transcript_28_HummViewer.txt'
# facts = facts_store[case]
    lean_facts = {
        'facts': facts['facts'],
        'product_description': facts['product_description']
    }
    if pitches[case] != "":
        # Reset for next run
        orcheditor.logs = []
        orcheditor.agents = {}
    try:
        pitch = orcheditor.orchestrate_with_edit(goal, lean_facts, verbose=True)
        pitches[case] = pitch
        metrics[case] = orcheditor.logs
        time_stamps[case] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    except Exception as e:
        print(f"{case} error: {e}")
        pitches[case] = ""
        metrics[case] = []
        time_stamps[case] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Reset for next run
    orcheditor.logs = []
    orcheditor.agents = {}

Generating pitches:   0%|          | 0/119 [00:00<?, ?it/s]

	>>> Subtasks: [{'name': 'Research and Structure the Pitch', 'description': 'Use DuckDuckGo to research effective pitch structures for similar products and organize the key points from the facts into a logical flow.', 'assigned_tools': ['duckduckgo']}, {'name': 'Enhance Pitch with Emotional Appeal', 'description': "Use the calculator to quantify the value proposition (e.g., ROI, market potential) and craft a narrative that highlights the product's unique benefits and customer loyalty.", 'assigned_tools': ['calculator']}]
	>>> Agent Outputs: {'Research and Structure the Pitch': '', 'Enhance Pitch with Emotional Appeal': 'To calculate the potential revenue based on the customer base and conversion rate:\n\n```json\n{\n  "name": "multiply",\n  "arguments": {\n    "a": 14000,\n    "b": 0.066\n  }\n}\n```\n\nThis will calculate the number of customers likely to purchase the product. The result can then be used to calculate potential'}


ERROR    Function multiply not found

	>>> Pitch Draft: {
    "Pitch": "Welcome, esteemed investors, to an extraordinary opportunity to be part of a revolutionary venture poised for exponential growth. Our company is at the forefront of innovation, addressing a critical need in the market with a unique solution that promises significant returns on investment. By joining us at this pivotal moment, you will not only be contributing to a transformative vision but also be part of a journey that is set to redefine industry standards.",
    "Initial_Offer": {
        "Valuation": "Estimated company valuation: $10,000,000",
        "Equity_Offered": "Percentage of equity offered to investors: 7%",
        "Funding_Amount": "The amount of funding requested: $700,000",
        "Key_Terms": "Pre-money valuation: $9,300,000; Post-money valuation: $10,000,000; Investment amount: $700,000; Equity stake: 7%."
    }
}
	>>> New goal: Edit a pitch with the given facts. With this pitch: {
    "Pitch": "Welcome, esteemed investors, to an ext

	>>> Subtasks: [{'name': 'Integrate Specific Metrics', 'description': 'Use the provided facts to add concrete metrics to the pitch, such as sales figures, profitability, customer base size, and product specifics. This will make the pitch more credible and data-driven.', 'assigned_tools': ['calculator']}, {'name': 'Clarify Investment Use', 'description': 'Outline how the $700,000 investment will be utilized, tying it to specific growth initiatives supported by the facts, such as expanding production, increasing marketing efforts, or improving customer acquisition strategies.', 'assigned_tools': ['duckduckgo']}, {'name': 'Revise Promotional Language', 'description': "Tone down overly promotional language and replace it with a more professional and factual presentation of the company's unique value proposition and growth potential.", 'assigned_tools': []}]


...

[00:52<00:52] 1/2 | Running Pitch Editing Iterations:  50%|█████      52.42s/it
Generating pitches:  99%|█████████▉| 118/119 [1:23:22<00:40, 40.14s/it]

Parsing Synthesizer Error: 1 validation error for SysthesizerResponse
  Invalid JSON: key must be a string at line 1 column 2 [type=json_invalid, input_value='{**All33: Revolutionizin... product in the market.', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/json_invalid
Response:


**All33: Revolutionizing Posture and Spinal Movement with Sit-In-Motion Technology**

All33 introduces a groundbreaking innovation in seating with its patented Sit-In-Motion Technology. This technology is designed to keep your spine moving and aligned naturally, just as it would if you were standing. The chair features a unique "chair within a chair" design that gently rocks, encouraging continuous movement of all 33 vertebrae. This subtle motion helps maintain perfect posture without any effort, providing a sitting experience that feels as natural as standing.

By promoting active spinal movement, the Sit-In-Motion Technology addresses common issues associated with pro

	>>> Subtasks: [{'name': 'Outline the pitch structure', 'description': 'Organize the key points and structure of the pitch based on the given facts.', 'assigned_tools': []}, {'name': 'Research effective pitching techniques', 'description': 'Use DuckDuckGo to find examples of successful pitches in the functional beverage market to inspire the tone and style.', 'assigned_tools': ['duckduckgo']}, {'name': 'Calculate and emphasize key metrics', 'description': 'Use the calculator to highlight important numerical data such as market size and production numbers in the pitch.', 'assigned_tools': ['calculator']}]
	>>> Agent Outputs: {'Outline the pitch structure': "<think>\nAlright, I need to outline the pitch structure based on the given facts. Let me first go through the facts provided. \n\nThe product is called Electra, a functional sports drink. It's targeted towards everyday athletes, hustlers, and doers. The market size is $23 billion, which is a significant number. It was launched 10 day

[00:22<?] 0/2 | Running Pitch Editing Iterations:   0%|           ?it/s
Generating pitches: 100%|██████████| 119/119 [1:23:44<00:00, 42.23s/it]

Parsing Synthesizer Error: 1 validation error for SysthesizerResponse
  Invalid JSON: key must be a string at line 1 column 2 [type=json_invalid, input_value="{**Pitch Structure Outli...s and market potential.", input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/json_invalid
Response:


**Pitch Structure Outline:**

1. **Opening Hook:**  
   - Highlight the $23 billion market size and Electra's recent launch (10 days ago) as a new entrant.

2. **Product Introduction:**  
   - Name: Electra  
   - Type: Functional sports drink  
   - Target Audience: Everyday athletes, hustlers, and doers

3. **Key Features:**  
   - Better-for-you hydration  
   - No artificial colors or ingredients  
   - Low sugar content  
   - High potassium relative to calories

4. **Manufacturing Details:**  
   - Made in Coppell, Texas  
   - Initial production run: 20,000 to 30,000 units

5. **Call to Action:**  
   - Encourage potential customers to try the product with a moti

In [8]:
calculated_metrics = []
for product, metric in metrics.items():
    calculated_metrics.append(metrics_calculation(metric))
input_lengths, output_lengths, latencies = zip(*calculated_metrics)

In [9]:
timestamp = datetime.now().strftime('%Y-%d-%m')
framework = "orchestrator"
layer = "orchestrator_1"
pitches_df = pd.DataFrame(
    data={
        'scenario_name':list(pitches.keys()),
        'framework':len(pitches) * [framework],
        'layer':len(pitches) * [layer],
        'model_name': len(pitches) * ["groq/" + orcheditor.orchestrator + "/groq" + orcheditor.editor],
        'model_identity':len(pitches) * ["groq/" + orcheditor.orchestrator + "/groq" + orcheditor.editor],
        'latency':latencies,
        'input_length':input_lengths,
        'output_length':output_lengths,
        'time_stamp':list(time_stamps.values()),
        'response':list(pitches.values()),
    }
)

In [10]:
pitches_df.to_excel(f"{timestamp}-orchestrator-editRAGlong-pitches-00.xlsx")